# Imports and set-up

In [3]:
import requests
import json
from dotenv import load_dotenv
import os
import uuid
import rsa
import base64

In [ ]:
load_dotenv()

token = os.environ.get("WISE_API_KEY")

wise_base_url = "https://api.wise.com"

private_key_path = "private_rsa.pem"

# Checking all "profiles" we have in Wise

In [ ]:
# Request headers
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# Send the GET request
response = requests.get(wise_base_url + "/v2/profiles", headers=headers)

# Print response
print(f"Status code: {response.status_code}")
if response.status_code == 200:
    profiles = response.json()
    print("Your profiles:")
    for profile in profiles:
        print(f"Profile ID: {profile['id']}")
        print(f"Profile Type: {profile['type']}")
        print(f"Business Name: {profile.get('businessName')}")
        print("-" * 30)
else:
    print(f"Error: {response.text}")

Check that the profile id below corresponds to the business account in Wise that you want to test with

In [ ]:
profile_id = profiles[-1]["id"]
profile_id

# Check your balance

In [ ]:
url = f"{wise_base_url}/v4/profiles/{profile_id}/balances?types=STANDARD"

response = requests.get(url, headers=headers)

print(f"Status code: {response.status_code}")
amount_currency_id = []
if response.status_code == 200:
    balances = response.json()
    for bal in balances:
        currency = bal["currency"]
        amount = bal["amount"]["value"]
        ida = bal["id"]
        print(f"{currency}: {amount} | id: {ida}")
        amount_currency_id.append([amount, currency, ida])
else:
    print(f"Error: {response.text}")
amount_currency_id.sort()

We will choose the account with the highest balance as the source account/ source currency

In [ ]:
_, source_currency, source_account_id = amount_currency_id[-1]

In [ ]:
source_currency

In [ ]:
source_account_id

# Quote ID

The first step in initiating a money transfer is producing a "quote". This will include information about fees and conversion rates between currencies (as well as how the transfer can be funded). We will produce a quote for transferring 2.01 worth of GBP from the source currency to GBP.

In [ ]:
# API endpoint
url = "{wise_base_url}/v3/profiles/{profile_id}/quotes".format(wise_base_url=wise_base_url, profile_id=profile_id)

# Request payload
payload = {
    "sourceCurrency": source_currency,
    "targetCurrency": "GBP",
    "sourceAmount": None,
    "targetAmount": 2.01,
    "payOut": "BANK_TRANSFER",
    "preferredPayIn": "BALANCE"
}

# Send the POST request
quote_response = requests.post(url, headers=headers, data=json.dumps(payload))

# Print response
print(f"Status code: {quote_response.status_code}")
if quote_response.status_code == 200:
    quote_id = quote_response.json()["id"]
else:
    print(f"Error: {quote_response.text}")

And that's how much the transfer will cost us (assuming we fund it from our balance, in the source currency)

In [ ]:
print(quote_response.json()["paymentOptions"][0]["sourceAmount"], source_currency)

Create a recipient -- in this case -- the author of this guide :)

In [ ]:
api_endpoint = "{wise_base_url}/v1/accounts".format(wise_base_url=wise_base_url, profile_id=profile_id)
payload = {
          "currency": "GBP",
          "type": "sort_code",
          "profile": profile_id,
          "accountHolderName": "Adam Kurkiewicz",
           "details": {
              "legalType": "PRIVATE",
              "sortCode": "040004",
              "accountNumber": "75283076",
              "dateOfBirth": "1961-01-01"
           }
         }

create_recipient_response = requests.post(api_endpoint, headers=headers, data=json.dumps(payload))
create_recipient_response

In [ ]:
recipient_id = create_recipient_response.json()["id"]
recipient_id

# Initiate the transfer

In [ ]:
existing_quote_id = quote_id
recipient_account_id = recipient_id  

# Create the transfer
transfer_url = "{wise_base_url}/v1/transfers".format(wise_base_url = wise_base_url)
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# Generate a unique transaction ID using uuid
unique_transaction_id = str(uuid.uuid4())

transfer_payload = {
    "targetAccount": recipient_account_id,
    "quoteUuid": existing_quote_id,
    "customerTransactionId": unique_transaction_id,  # Using a simple UUID as the transaction ID
    "details": {
        "reference": "tip",
        "transferPurpose": "Tipping the author of a useful guide on making payments with Python",
        "sourceOfFunds": "other"
    }
}

# Send the transfer request
transfer_response = requests.post(transfer_url, headers=headers, data=json.dumps(transfer_payload))

# Print results
print(f"Status code: {transfer_response.status_code}")
if transfer_response.status_code in [200, 201]:
    transfer_data = transfer_response.json()
    print(f"Transfer created with ID: {transfer_data.get('id')}")
    print(json.dumps(transfer_data, indent=2))
    transfer_id = transfer_data["id"]
else:
    print(f"Error creating transfer: {transfer_response.text}")

As you can see above, we will be sending the following amount to the recipient:

In [ ]:
print(transfer_data["targetValue"], transfer_data["targetCurrency"])

In [ ]:
def do_sca_challenge(one_time_token):
    print('doing sca challenge')

    # Read the private key file as bytes.
    with open(private_key_path, 'rb') as f:
        private_key_data = f.read()

    private_key = rsa.PrivateKey.load_pkcs1(private_key_data, 'PEM')

    # Use the private key to sign the one-time-token that was returned 
    # in the x-2fa-approval header of the HTTP 403.
    signed_token = rsa.sign(
        one_time_token.encode('ascii'), 
        private_key, 
        'SHA-256')

    # Encode the signed message as friendly base64 format for HTTP 
    # headers.
    signature = base64.b64encode(signed_token).decode('ascii')

    return signature

def fund_transfer(transfer_id, one_time_token="", signature=""):
    """
    Fund a transfer using the existing balance, with SCA support.
    """
    funding_url = f"{wise_base_url}/v3/profiles/{profile_id}/transfers/{transfer_id}/payments"
    
    funding_payload = {
        "type": "BALANCE"
    }
    
    headers = {
        'Authorization': 'Bearer ' + token,
        'User-Agent': 'tw-funding-sca',
        'Content-Type': 'application/json'
    }
    
    # Add SCA headers if we have them
    if one_time_token != "":
        headers['x-2fa-approval'] = one_time_token
        headers['X-Signature'] = signature
        print(f"Using SCA headers: {one_time_token}, {signature}")
    
    print(f"POST {funding_url}")

    print("HEADERS", headers)

    print("BODY", json.dumps(funding_payload).encode('utf-8'))
    
    r = requests.post(
        funding_url, 
        data=json.dumps(funding_payload),
        headers=headers
    )
    
    print(f"Status: {r.status_code}")
    
    if r.status_code == 200 or r.status_code == 201:
        return r.json()
    elif r.status_code == 403 and r.headers['x-2fa-approval'] is not None:
        # Handle SCA challenge
        one_time_token = r.headers['x-2fa-approval']
        print(f"SCA challenge required, token: {one_time_token}")
        signature = do_sca_challenge(one_time_token)
        return fund_transfer(transfer_id, one_time_token, signature)
    else:
        print(f"Failed: {r.status_code}")
        print(r.json())
        raise Exception(f"Failed to fund transfer: {r.json()}")

# Let's tip the author of this guide!

In [ ]:
fund_transfer(transfer_id)

If everything worked well, you should get a json showing something like:

```{'type': 'BALANCE',
 'status': 'COMPLETED',
 'errorCode': None,
 'errorMessage': None,
 'balanceTransactionId': 4047687055}
```